Tạo 2 file references: 
* chebi_reference.tsv
* uniprot_reference.tsv

Tạo 2 file query sets:
* chebi_query_set.tsv
* uniprot_query_set.tsv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# for reading obo file
!pip install obonet

GET CHEBI REFERENCE

In [ ]:
import networkx
import obonet
import re
# Read the taxrank ontology
url = "/content/drive/MyDrive/chebi.obo"
graph = obonet.read_obo(url)

chebi_reference_set = {}
for node in graph.nodes(data=True):
  if node[1].get('subset') is not None:
    if node[1].get('subset')[0] == '3_STAR':
      
      terms = []
      
      id = node[0].split(':')[1]
      name = node[1].get('name')

      terms.append(name)
      if name is None: 
        continue

      is_obsolete = node[1].get('is_obsolete')
      if(is_obsolete is not None):
        print(id)
      ls = node[1].get('synonym')
      if ls is not None:
        for synonym in ls:
          term = re.findall(r'["](.*?)["]',synonym)
          if len(term) > 0:
            term = term[0]        
            if term != name:
              if term == "" or term is None:
                print('y')
              terms.append(term)

      str_terms = ""
      for term in terms:
        if term is None:
          print('y')
        str_terms = str_terms + term + '\t'

      str_terms = str_terms[:-1]
      chebi_reference_set[id] = str_terms


In [ ]:
len(chebi_reference_set)

58597

In [ ]:
chebi_reference_set

GET UNIPROT REFERENCE

In [ ]:
myFile= open("/content/drive/My Drive/NSEEN/uniprot-reviewed_yes.tab")
uniprot_reference_set = {}
for aRow in myFile:
  aRow = list(aRow.split('\t'))  
  
  terms = aRow[3].split(' (')
  for i,tmp in enumerate(terms):
    if tmp[-1] == ')':
      terms[i] = tmp[:-1]
  
  str_terms = ""

  for term in terms:
    str_terms = str_terms + term + '\t'
  str_terms = str_terms[:-1]
  uniprot_reference_set[aRow[0]] = str_terms 

myFile.close()

GET QUERY SET

In [ ]:
import xml.etree.ElementTree as ET
import os
trees = []; #id in these file is int

dir = '/content/drive/My Drive/NSEEN/BioIDtraining_2/caption_bioc'
training_data = []
for fname in sorted(os.listdir(dir)): 
  if fname[-4:] == '.xml' and fname[0:2] != '._': 
    trees.append(ET.parse(os.path.join(dir,fname)))
    
for tree in trees: 
  root = tree.getroot() 

  for document in root.iter('document'):
      for passage in document.iter('passage'):
        for annotation in passage.iter('annotation'):
          term_type = annotation[0].text
          if "CHEBI" in term_type or "Uniprot" in term_type:
            text = annotation[-1].text

            term = {}
            term[text] = term_type
            training_data.append(term)

In [ ]:
chebi_query_set = {}

# 1 term có thể thuộc nhiều ID
for term in training_data:
  for key, value in term.items():
    if 'CHEBI' in value:
      IDs = value.split('|')
      for i,ID in enumerate(IDs):
        IDs[i] = ID[ID.find(':')+1:]

      str_ID = ""
      for ID in IDs:
        str_ID = str_ID + ID + '\t'
      str_ID = str_ID[:-1]
      chebi_query_set[key] = str_ID

In [ ]:
uniprot_query_set = {}
for term in training_data:
  for key, value in term.items():
    if 'Uniprot' in value:
      IDs = value.split('|')
      for i,ID in enumerate(IDs):
        IDs[i] = ID[ID.find(':')+1:]

      str_ID = ""
      for ID in IDs:
        str_ID = str_ID + ID + '\t'
      
      str_ID = str_ID[:-1]
      
      uniprot_query_set[key] = str_ID

RESULT

In [ ]:
print('Kich thuoc chebi_reference_set: ', len(chebi_reference_set))
print('Kich thuoc uniprot_reference_set: ', len(uniprot_reference_set))
print('Kich thuoc chebi_query_set: ', len(chebi_query_set))
print('Kich thuoc uniprot_query_set: ', len(uniprot_query_set))

Kich thuoc chebi_reference_set:  136609
Kich thuoc uniprot_reference_set:  563973
Kich thuoc chebi_query_set:  1296
Kich thuoc uniprot_query_set:  2746


SAVE RESULT

In [ ]:
def save_dict_to_file(dictionary, dir):
  with open(dir, 'w') as f:
    for key in dictionary.keys():
      f.write("%s: %s\n" % (key,  dictionary[key]))

In [ ]:
save_dict_to_file(chebi_reference_set, '/content/drive/My Drive/chebi_reference_set.tsv')


In [ ]:
save_dict_to_file(chebi_reference_set, '/content/drive/My Drive/chebi_reference_set.tsv')
save_dict_to_file(uniprot_reference_set, '/content/drive/My Drive/uniprot_reference_set.tsv')
save_dict_to_file(chebi_query_set, '/content/drive/My Drive/chebi_query_set.tsv')
save_dict_to_file(uniprot_query_set, '/content/drive/My Drive/uniprot_query_set.tsv')

In [ ]:
count = 0
for key in chebi_query_set.keys():
  IDs = chebi_query_set[key]
  IDs = IDs.split('\t')

  for ID in IDs:
    if ID in chebi_reference_set.keys():
      if key in chebi_reference_set[ID]:
        count += 1
        break
print(count)

634


##EDA

####CHEBI

In [ ]:
IDs = []
n_matchstring = []
for key,values in chebi_query_set.items():
  ID_arr = values.split('\t')
  for ID in ID_arr:
    IDs.append(ID)

In [ ]:
print('Number of query is: ', len(chebi_query_set))
print('Number of unique query is:', len(set(IDs)))

Number of query is:  1296
Number of unique query is: 748


In [ ]:
n_match = 0
for key in chebi_query_set.keys():
  for values in chebi_reference_set.values():
    values = values.split('\t')
    flag = False
    for value in values:
      if value == key:
        n_match += 1
        flag = True
        break
    if flag == True:
      break
print(n_match)

673
